# Feed-forward neural network model (multivariate output)

In this notebook, we:
- prepare time series data for training a feed-forward neural network forecasting model
- get data in the required shape for the keras API
- implement a feed-forward NN model in keras to predict the next 3 steps ahead (time *t+1* to *t+3*) in the time series. This model uses recent values of temperature and load as the model input. The model will be trained to output a vector, the elements of which are ordered predictions for future time steps.
- enable early stopping to reduce the likelihood of model overfitting
- evaluate the model on a test dataset




Please run this notebook after completing 0_data_setup notebook.

In [ ]:
import sys
sys.path.append('..')
import os
import warnings
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
from glob import glob
from collections import UserDict
from common.utils import *
from IPython.display import Image
%matplotlib inline

pd.options.display.float_format = '{:,.2f}'.format
np.set_printoptions(precision=2)
warnings.filterwarnings("ignore")

Load data into Pandas dataframe

In [ ]:
data_dir = '../data/'
energy = load_data(data_dir)
energy.head()

In [ ]:
valid_start_dt = '2014-09-01 00:00:00'
test_start_dt = '2014-11-01 00:00:00'

## Data preparation - training set

For this example, we will set T=6. This means that the input for each sample is a vector of the prevous 6 hours of the energy load. The choice of T=6 was arbitrary but should be selected through experimentation.  

HORIZON=3 specifies that we have a forecasting horizon of 3 (t+1, t+2, t+3)

In [ ]:
Image('../images/multistep_forecast.png')

In [ ]:
T = 6
HORIZON = 3

In [ ]:
train = energy.copy()[energy.index < valid_start_dt][['load', 'temp']]

In [ ]:
from sklearn.preprocessing import MinMaxScaler

y_scaler = MinMaxScaler()
y_scaler.fit(train[['load']])

X_scaler = MinMaxScaler()
train[['load', 'temp']] = X_scaler.fit_transform(train)

Use the TimeSeriesTensor convenience class to:
1. Shift the values of the time series to create a Pandas dataframe containing all the data for a single training example
2. Discard any samples with missing values
3. Transform this Pandas dataframe into a numpy array of shape (samples, features) for input into Keras

The class takes the following parameters:

- **dataset**: original time series
- **target** name of the target column
- **H**: the forecast horizon
- **tensor_structure**: a dictionary discribing the tensor structure in the form { 'tensor_name' : (range(max_backward_shift, max_forward_shift), [feature, feature, ...] ) }
- **freq**: time series frequency (default 'H' - hourly)
- **drop_incomplete**: (Boolean) whether to drop incomplete samples (default True)

In [ ]:
tensor_structure = {'X':(range(-T+1, 1), ['load', 'temp'])}
train_inputs = TimeSeriesTensor(train, 'load', HORIZON, tensor_structure)

In [ ]:
train_inputs.dataframe.head(3)

In [ ]:
X_train = train_inputs.dataframe.as_matrix()[:,HORIZON:]
X_train.shape

In [ ]:
train_inputs['target']

## Data preparation - validation set

Construct validation set (keeping W hours from the training set in order to construct initial features)

In [ ]:
look_back_dt = dt.datetime.strptime(valid_start_dt, '%Y-%m-%d %H:%M:%S') - dt.timedelta(hours=T-1)
valid = energy.copy()[(energy.index >=look_back_dt) & (energy.index < test_start_dt)][['load', 'temp']]
valid[['load', 'temp']] = X_scaler.transform(valid)
valid_inputs = TimeSeriesTensor(valid, 'load', HORIZON, tensor_structure)
X_valid = valid_inputs.dataframe.as_matrix()[:,HORIZON:]
X_valid.shape

## Implement feed-forward neural network

We implement feed-forward neural network with the following structure:

In [ ]:
Image('../images/ff_multi_step_vector_output.png')

In [ ]:
from keras.models import Model, Sequential
from keras.layers import GRU, Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
LATENT_DIM = 5
BATCH_SIZE = 32
EPOCHS = 50

In [ ]:
model = Sequential()
model.add(Dense(LATENT_DIM, activation="relu", input_shape=(2*T,)))
model.add(Dense(HORIZON))

In [ ]:
model.compile(optimizer='RMSprop', loss='mse')

In [ ]:
model.summary()

Specify the early stopping criteria. We monitor the validation loss (in this case the mean squared error) on the validation set after each training epoch. If the validation loss has not improved by min_delta after patience epochs, we stop the training.

In [ ]:
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5)

In [ ]:
best_val = ModelCheckpoint('model_{epoch:02d}.h5', save_best_only=True, mode='min', period=1)

In [ ]:
history = model.fit(X_train,
                    train_inputs['target'],
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_data=(X_valid, valid_inputs['target']),
                    callbacks=[earlystop, best_val],
                    verbose=1)

## Evaluate the model

Load the model with the smallest mape

In [ ]:
best_epoch = np.argmin(np.array(history.history['val_loss']))+1
model.load_weights("model_{:02d}.h5".format(best_epoch))

In [ ]:
look_back_dt = dt.datetime.strptime(test_start_dt, '%Y-%m-%d %H:%M:%S') - dt.timedelta(hours=T-1)
test = energy.copy()[test_start_dt:][['load', 'temp']]
test[['load', 'temp']] = X_scaler.transform(test)
test_inputs = TimeSeriesTensor(test, 'load', HORIZON, tensor_structure)
X_test = test_inputs.dataframe.as_matrix()[:,HORIZON:]
X_test.shape

In [ ]:
predictions = model.predict(X_test)

In [ ]:
predictions

In [ ]:
eval_df = create_evaluation_df(predictions, test_inputs, HORIZON, y_scaler)
eval_df.head()

Compute MAPE for each forecast horizon

In [ ]:
eval_df['APE'] = (eval_df['prediction'] - eval_df['actual']).abs() / eval_df['actual']
eval_df.groupby('h')['APE'].mean()

Compute MAPE across all predictions

In [ ]:
mape(eval_df['prediction'], eval_df['actual'])

Plot actuals vs predictions at each horizon for first week of the test period. As is to be expected, predictions for one step ahead (*t+1*) are more accurate than those for 2 or 3 steps ahead

In [ ]:
plot_df = eval_df[(eval_df.timestamp<'2014-11-08') & (eval_df.h=='t+1')][['timestamp', 'actual']]
for t in range(1, HORIZON+1):
    plot_df['t+'+str(t)] = eval_df[(eval_df.timestamp<'2014-11-08') & (eval_df.h=='t+'+str(t))]['prediction'].values

fig = plt.figure(figsize=(15, 8))
ax = plt.plot(plot_df['timestamp'], plot_df['actual'], color='red', linewidth=4.0)
ax = fig.add_subplot(111)
ax.plot(plot_df['timestamp'], plot_df['t+1'], color='blue', linewidth=4.0, alpha=0.75)
ax.plot(plot_df['timestamp'], plot_df['t+2'], color='blue', linewidth=3.0, alpha=0.5)
ax.plot(plot_df['timestamp'], plot_df['t+3'], color='blue', linewidth=2.0, alpha=0.25)
plt.xlabel('timestamp', fontsize=12)
plt.ylabel('load', fontsize=12)
ax.legend(loc='best')
plt.show()

clean up model files

In [ ]:
for m in glob('model_*.h5'):
    os.remove(m)